# Project 1 - Part2

# Data Cleaning

In [1]:
# import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load data
file = r"C:\Users\kbundy\Documents\Coding\CodingDojo\food-sales-predictions-kb\sales_predictions.csv"

df = pd.read_csv(file)

# dataframe preview
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


## Dataframe Info

### Dataframe Observations Notes
 * 8523 entries
 * two ID columns, no nulls, but not unique
 * nulls are in two columns (item_weight is numeric and outlet_size is categorical)
 * outlet_type column can possibly be expanded
 * duplicates exist in categorical columns
 * irregular spellings in item_fat_content

### Irregular Spelling
#### Item_Fat_Content Column  
 * normalize category spellings
 * use `.replace()`

In [4]:
# examine Item_Fat_Content column elements
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [5]:
# replace elements to normalize
df['Item_Fat_Content'].replace(['LF','low fat'],'Low Fat',inplace=True)
df['Item_Fat_Content'].replace('reg','Regular',inplace=True)

In [6]:
# verify replacements
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

## Missing Values Quick Look
The approach to fill in missing data can be determined by exploring relationships within the data

In [7]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

### Explore data elements to see relationship between item_weight and item_identifier  
 * item_identifier and item_weight seem to have a 1 to 1 correlation
 * fill NaN with mode of group

In [8]:
# filter dataframe
df.loc[df['Item_Identifier'] == 'DRA59',['Item_Identifier','Item_Weight']]

,Item_Identifier,Item_Weight
1876,DRA59,8.27
3927,DRA59,8.27
3962,DRA59,8.27
5609,DRA59,8.27
6057,DRA59,NaN
6179,DRA59,NaN
6338,DRA59,8.27
8371,DRA59,8.27


### Explore relationship between outlet_identifier and outlet_size  
 Outlet_size is missing from 3 outlet_identifiers
  * OUT010, OUT017, OUT045
  * Outlet_Size varies with Outlet_Type and Outlet_Location_Type
  * did not observe any relationship useful to fill NaN in Outlet_Size column

In [9]:
# groupby outlet_identifier
df.groupby('Outlet_Identifier').count().sort_values('Outlet_Size')

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
Outlet_Identifier,,,,,,,,,,,
OUT010,555,555,555,555,555,555,555,0,555,555,555
OUT017,926,926,926,926,926,926,926,0,926,926,926
OUT045,929,929,929,929,929,929,929,0,929,929,929
OUT019,528,0,528,528,528,528,528,528,528,528,528
OUT018,928,928,928,928,928,928,928,928,928,928,928
OUT035,930,930,930,930,930,930,930,930,930,930,930
OUT046,930,930,930,930,930,930,930,930,930,930,930
OUT049,930,930,930,930,930,930,930,930,930,930,930
OUT013,932,932,932,932,932,932,932,932,932,932,932


In [10]:
# groupby outlet location type to check if Tiers and Outlet size are related
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Size','Outlet_Identifier']).size()

Outlet_Location_Type  Outlet_Type        Outlet_Size  Outlet_Identifier
Tier 1                Grocery Store      Small        OUT019               528
                      Supermarket Type1  Medium       OUT049               930
                                         Small        OUT046               930
Tier 2                Supermarket Type1  Small        OUT035               930
Tier 3                Supermarket Type1  High         OUT013               932
                      Supermarket Type2  Medium       OUT018               928
                      Supermarket Type3  Medium       OUT027               935
dtype: int64

In [11]:
# groupby outlet_type
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Identifier'])['Item_Outlet_Sales'].sum()

Outlet_Location_Type  Outlet_Type        Outlet_Identifier
Tier 1                Grocery Store      OUT019               1.796941e+05
                      Supermarket Type1  OUT046               2.118395e+06
                                         OUT049               2.183970e+06
Tier 2                Supermarket Type1  OUT017               2.167465e+06
                                         OUT035               2.268123e+06
                                         OUT045               2.036725e+06
Tier 3                Grocery Store      OUT010               1.883402e+05
                      Supermarket Type1  OUT013               2.142664e+06
                      Supermarket Type2  OUT018               1.851823e+06
                      Supermarket Type3  OUT027               3.453926e+06
Name: Item_Outlet_Sales, dtype: float64

In [12]:
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Size','Outlet_Identifier'])['Item_Outlet_Sales'].sum()

Outlet_Location_Type  Outlet_Type        Outlet_Size  Outlet_Identifier
Tier 1                Grocery Store      Small        OUT019               1.796941e+05
                      Supermarket Type1  Medium       OUT049               2.183970e+06
                                         Small        OUT046               2.118395e+06
Tier 2                Supermarket Type1  Small        OUT035               2.268123e+06
Tier 3                Supermarket Type1  High         OUT013               2.142664e+06
                      Supermarket Type2  Medium       OUT018               1.851823e+06
                      Supermarket Type3  Medium       OUT027               3.453926e+06
Name: Item_Outlet_Sales, dtype: float64

## Fill Missing Values

#### Use fill method to replace missing data
 * Item_Weight can be filled with mode based on Item_identifier
 * Outlet_Size can be filled with mode based on Outlet_Identifier

### Outlet Weight Column

In [13]:
df['Item_Weight'] = df.groupby('Item_Identifier')['Item_Weight'].apply(lambda x: x.fillna(x.mean()))

In [14]:
df['Item_Weight'].isnull().sum()

4

In [15]:
null_weight = df['Item_Weight'].isnull()
df[null_weight]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


In [16]:
# cannot infer the remaining 4 missing weight values
# drop those rows
df.dropna(axis=0, subset=['Item_Weight'], inplace=True)

In [17]:
# verify dropna 
df['Item_Weight'].isnull().sum()

0

### Outlet Size Column

In [18]:
# check missing value counts by Outlet Identifier
nafilt = df['Outlet_Size'].isna()
df[nafilt][['Outlet_Identifier','Outlet_Location_Type','Outlet_Type']].value_counts()

Outlet_Identifier  Outlet_Location_Type  Outlet_Type      
OUT045             Tier 2                Supermarket Type1    929
OUT017             Tier 2                Supermarket Type1    926
OUT010             Tier 3                Grocery Store        555
dtype: int64

In [19]:
# cannot infer Outlet Size for OUT010, OUT017, OUT045
# drop rows, 2413 observations
df.dropna(axis=0, subset=['Outlet_Size'], inplace=True)
df.reset_index(drop=True)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
4,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
...,...,...,...,...,...,...,...,...,...,...,...,...
6104,FDF53,20.750,Regular,0.083607,Frozen Foods,178.8318,OUT046,1997,Small,Tier 1,Supermarket Type1,3608.6360
6105,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
6106,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
6107,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6109 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6109 non-null   object 
 1   Item_Weight                6109 non-null   float64
 2   Item_Fat_Content           6109 non-null   object 
 3   Item_Visibility            6109 non-null   float64
 4   Item_Type                  6109 non-null   object 
 5   Item_MRP                   6109 non-null   float64
 6   Outlet_Identifier          6109 non-null   object 
 7   Outlet_Establishment_Year  6109 non-null   int64  
 8   Outlet_Size                6109 non-null   object 
 9   Outlet_Location_Type       6109 non-null   object 
 10  Outlet_Type                6109 non-null   object 
 11  Item_Outlet_Sales          6109 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 620.4+ KB


### Expand Outlet_Type Column  
 * is this a necessary step...???
 * create column Outlet_Category
 * create column Outlet_Type

# Save File to CSV

In [21]:

filename = r"C:\Users\kbundy\Documents\Coding\CodingDojo\food-sales-predictions-kb\sales_predictions_clean.csv"

df.to_csv(filename, index=False)